In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
import matplotlib.image as mpimg
import glob, os
from pathlib import Path
from IPython.core.display import display,HTML
pd.set_option('display.max_colwidth', None)

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
labels = pd.read_csv('datasets/mikey-devon-labels_validation-221109.csv')

In [19]:
a = labels['tag_list'].str.contains(",")
labels['multiple_tags'] = a
labels.loc[labels['multiple_tags'] == True]

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,multiple_tags
1114,Devon Snyder,2528,487,University District,CurbRamp,47.659523,-122.319687,jCGaqx_i5lllqbp3q1GWUw,2022-11-09 08:03:27.714-08,NaN,3.0,f,"missing tactile warning,points into traffic,narrow",NaN,NaN,NaN,NaN,True
1115,Devon Snyder,2529,488,University District,CurbRamp,47.659401,-122.319725,z9UEjhLFWd_yO6TmMzABfA,2022-11-09 08:04:02.134-08,NaN,2.0,f,"points into traffic,missing tactile warning",NaN,NaN,NaN,NaN,True
1116,Devon Snyder,2530,476,University District,CurbRamp,47.659454,-122.319939,ndtOrb8y8Cnt1HpvVXWNuQ,2022-11-09 08:04:41.577-08,NaN,2.0,f,"narrow,missing tactile warning",NaN,NaN,NaN,NaN,True
1118,Devon Snyder,2532,489,University District,CurbRamp,47.659496,-122.319839,jCGaqx_i5lllqbp3q1GWUw,2022-11-09 08:07:11.519-08,NaN,3.0,f,"points into traffic,narrow,missing tactile warning",NaN,NaN,NaN,NaN,True
1120,Devon Snyder,2534,479,University District,SurfaceProblem,47.659721,-122.319923,SRZ-Py452sb8rEJfkHzHWw,2022-11-09 08:08:26.318-08,NaN,2.0,f,"cracks,uneven/slanted",NaN,NaN,NaN,NaN,True


In [7]:
#create labels geo dataframe
points = labels.apply(lambda row: Point(row.lng, row.lat), axis=1)
labels_geo = gpd.GeoDataFrame(labels, geometry=points)
labels_geo.crs = {'init': 'epsg:4326'}
labels_geo

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,time_created,correct,severity,temporary,tag_list,description,Unnamed: 14,Unnamed: 15,Unnamed: 16,geometry
0,Devon Snyder,3,480,University District,Obstacle,47.661041,-122.320709,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:36.787-07,t,3.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66104)
1,Devon Snyder,4,480,University District,Obstacle,47.661026,-122.320702,-L8Sgb0ozIqqanCBfAZjYQ,2022-05-10 14:00:52.367-07,t,2.0,f,pole,NaN,NaN,NaN,NaN,POINT (-122.32070 47.66103)
2,Devon Snyder,5,480,University District,SurfaceProblem,47.661060,-122.320709,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:01:21.608-07,t,2.0,f,bumpy,NaN,NaN,NaN,NaN,POINT (-122.32071 47.66106)
3,Devon Snyder,8,480,University District,Occlusion,47.661106,-122.320724,OH4yKYW98UY1aMK_b68Etw,2022-05-10 14:02:15.133-07,NaN,NaN,f,NaN,NaN,NaN,NaN,NaN,POINT (-122.32072 47.66111)
4,Devon Snyder,9,480,University District,Obstacle,47.661098,-122.320663,O0VZ-MF6Nrmjhkb2nCRhCw,2022-05-10 14:02:29.636-07,t,3.0,f,sign,temporary due to construction,NaN,NaN,NaN,POINT (-122.32066 47.66110)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,mikey,2493,810,Windermere,SurfaceProblem,47.672043,-122.267929,Z6BJD4bSdDiMxwMdls5oNw,2022-07-06 14:12:40.653-07,NaN,1.0,f,uneven/slanted,NaN,NaN,NaN,NaN,POINT (-122.26793 47.67204)
2021,mikey,2494,716,Windermere,Obstacle,47.671928,-122.268097,sOyfUL3XT8ETE-9XfeaXvQ,2022-07-06 14:14:34.691-07,NaN,1.0,f,vegetation,NaN,NaN,NaN,NaN,POINT (-122.26810 47.67193)
2022,mikey,2495,715,Windermere,NoSidewalk,47.672016,-122.268433,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:48.626-07,NaN,3.0,f,street has a sidewalk,NaN,NaN,NaN,NaN,POINT (-122.26843 47.67202)
2023,mikey,2496,716,Windermere,SurfaceProblem,47.671883,-122.268181,xbqjQfJSHA1O90JnBHQJGw,2022-07-06 14:14:55.957-07,NaN,1.0,f,cracks,NaN,NaN,NaN,NaN,POINT (-122.26818 47.67188)


In [8]:
# Seperate Seattle and Chicago data
chicago = labels_geo.loc[labels_geo['lng'] > -105]
seattle = labels_geo.loc[labels_geo['lng'] < -105]

In [11]:
map_clusters(chicago,0)

In [26]:
def agree_rate(label_data,i):
    df = cluster_label_type_at_index(label_data,i)[2]
    df["clustered"] = df.duplicated(subset="cluster_id", keep=False)
    df1 = df[df['clustered'] == True]
    
    if len(df) == 0:
        return 0
    else: 
        return len(df1)/len(df)

In [18]:
agree_rate(seattle,0)

0.8494623655913979

In [19]:
agree_rate(seattle,2)

0.5764075067024129

In [20]:
agree_rate(chicago,0)

0.9302325581395349

In [21]:
agree_rate(chicago,2)

0.5229357798165137

In [27]:
j=[]
k=[]
l=[]
for i in [0,1,2,7]:
     j.append(agree_rate(seattle,i))
for i in [0,1,2,7]:
    k.append(agree_rate(chicago,i))
for i in [0,1,2,7]:
    l.append( label_types[i])
agree_rate_df = pd.DataFrame(list(zip(l,j, k)),
              columns=['label_type','seattle', 'chicago'])

In [29]:
agree_rate_df

,label_type,seattle,chicago
0,CurbRamp,0.849462,0.930233
1,NoSidewalk,0.670807,0.666667
2,Problem,0.576408,0.522936
3,NoCurbRamp,0.852792,0.000000


In [31]:
agree_rate(chicago,4)

0.5

In [38]:
def agreement_df(label_index_list,gdf):
    j=[]
    k=[]
    l=[]
    for i in label_index_list:
        j.append(agree_rate(seattle,i))
    for i in label_index_list:
        k.append(agree_rate(chicago,i))
    for i in label_index_list:
        l.append( label_types[i])
    agree_rate_df = pd.DataFrame(list(zip(l,j, k)),columns=['label_type','seattle', 'chicago'])
    df = agree_rate_df.style.format({
    'seattle': '{:,.2%}'.format,
    'chicago': '{:,.2%}'.format,})
    
    return df 

In [36]:
round(agreement_df([0,1,2,4,5,7]),4)

,label_type,seattle,chicago
0,CurbRamp,0.8495,0.9302
1,NoSidewalk,0.6708,0.6667
2,Problem,0.5764,0.5229
3,SurfaceProblem,0.5587,0.5000
4,Obstacle,0.4110,0.5385
5,NoCurbRamp,0.8528,0.0000


In [39]:
agreement_df([0,1,2,4,5,7])

,label_type,seattle,chicago
0,CurbRamp,84.95%,93.02%
1,NoSidewalk,67.08%,66.67%
2,Problem,57.64%,52.29%
3,SurfaceProblem,55.87%,50.00%
4,Obstacle,41.10%,53.85%
5,NoCurbRamp,85.28%,0.00%
